In [ ]:
import os

from retriever import get_retriever
import json

In [2]:
file_path = "C:\Cods\ChatBot\docs\hc_articles.json"
embed_model_name = "thenlper/gte-small" 
llm_model_name = "HuggingFaceH4/zephyr-7b-beta"

In [3]:
my_retriever = get_retriever(file_path, embed_model_name)

c:\Users\Ugne\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "how to install NORDVPN?"
results = my_retriever.get_relevant_documents(query)
for result in results:
    print(result)
    print("---------------------")

page_content='{"url": "https://support.nordvpn.com/hc/en-us/articles/20289779952017-How-to-start-using-NordVPN", "title": "How to start using NordVPN", "name": "How to start using NordVPN", "body": "If you are interested in starting to use the NordVPN service but are unsure how to do so, this article will introduce you to the steps needed to begin using NordVPN today.\u00a0 Download the NordVPN application.\u00a0If you are not sure how to install and set up the application, choose the OS\u00a0you're going to use and follow the guidelines for the procedure: Install NordVPN on Windows 7 and 8.1\u00a0 Install NordVPN on Windows 10 and 11\u00a0 Install NordVPN on macOS\u00a0 Install NordVPN on Linux\u00a0 Install NordVPN on Android\u00a0  Install NordVPN on iOSIn these guides, you can learn about the features that NordVPN offers!\u00a0  After you have set up the application - you need to\u00a0purchase the desired plan.The following article will introduce you to\u00a0NordVPN Plans and Price

C:\Users\Ugne\AppData\Local\Temp\ipykernel_30504\3507462887.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = my_retriever.get_relevant_documents(query)


In [7]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")

In [11]:
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=1000,
)

Device set to use cuda:0


In [12]:
from langchain.prompts import ChatPromptTemplate

In [13]:
template = """You are a helpful assistant. Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
results2 = my_retriever.get_relevant_documents("What is NORDVPN?")

In [20]:
for thing in results2:
    print('----------')
    print(thing)
    
type(thing)

----------
page_content='{"url": "https://support.nordvpn.com/hc/en-us/articles/19685546104209-What-is-NordVPN", "title": "What is NordVPN", "name": "What is NordVPN", "body": "NordVPN is the gateway to a secure and private access to the internet. It works by enveloping all of your online activities\u00a0in\u00a0a layer of encryption, while also hiding information about\u00a0your virtual location. This way, you stay invisible to hackers, your ISP,\u00a0governmental agencies and other prying eyes lurking throughout the net. At the same time, your data, such as your bank credentials, is\u00a0protected from potential attacks.\u00a0 Why would you need NordVPN when... ...staying at home?\u00a0It's true that there\u00a0are fewer threats to your online security if you're using a personal, trusted network. However, your privacy is left vulnerable. Browser destinations, internet-using apps and other internet-related information is visible to your ISP. By connecting to a NordVPN server, the only

langchain_core.documents.base.Document

In [47]:
def get_body_content(document):
  
    string_text = document.page_content
    dat = json.loads(string_text)
    body_content = dat.get('body')
    body_title = dat.get('title')
    body_url = dat.get('url')
    return body_content, body_title, body_url

In [53]:
context = "\nExtracted documents:\n"
context = "\n".join([
    f"Document {i + 1}:\nTitle: {get_body_content(doc)[0]}\n{get_body_content(doc)[1]}\nUrl: {get_body_content(doc)[2]}"
    for i, doc in enumerate(results2)
])
print(context)

Document 1:
Title: NordVPN is the gateway to a secure and private access to the internet. It works by enveloping all of your online activities in a layer of encryption, while also hiding information about your virtual location. This way, you stay invisible to hackers, your ISP, governmental agencies and other prying eyes lurking throughout the net. At the same time, your data, such as your bank credentials, is protected from potential attacks.  Why would you need NordVPN when... ...staying at home? It's true that there are fewer threats to your online security if you're using a personal, trusted network. However, your privacy is left vulnerable. Browser destinations, internet-using apps and other internet-related information is visible to your ISP. By connecting to a NordVPN server, the only footprint left of your activity is that, well, you've connected to one of our servers — that's it. Moreover, if you have to deal with internet throttling, NordVPN will come in handy. ...going out? 